In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
track = pd.read_csv('../saved-files/threshold-all/all-cells-first_landfall-threshold-all.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,145,333.698552,288.034937,66,15,79,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.393699e+06,...,68.352148,14.895784,79,0 days 00:00:00,108.538869,13.245867,34.567482,187.081946,True,0
2,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,68.773477,15.900718,118,0 days 00:00:00,52.225055,13.245867,34.567482,127.366079,True,0
3,1,55,291.000000,279.391287,3,10,184,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,68.747008,14.775128,176,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0
4,1,92,362.373711,215.778017,21,10,202,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.422374e+06,...,68.137183,13.137425,192,0 days 00:00:00,168.913689,12.974534,35.577388,35.824467,False,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,295,115,277.072836,343.163451,8,20,35258,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.337073e+06,...,68.817054,16.352027,12240,0 days 00:05:00,49.219517,15.270663,40.109338,123.385353,True,0
2412,296,72,275.246502,319.000000,3,15,35289,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335247e+06,...,68.856887,15.770719,12273,0 days 00:00:00,43.768175,15.390894,40.106802,36.815098,True,0
2413,296,90,196.864798,399.000000,2,20,35301,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.256865e+06,...,69.479063,17.970580,12277,0 days 00:00:00,81.614861,15.390894,40.106802,22.883585,True,0
2414,297,110,286.700337,370.870819,19,20,35376,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.346700e+06,...,68.700466,16.993560,12294,0 days 00:00:00,72.031625,15.206985,38.598690,73.632575,True,0


In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[104.41324626907375,
 108.53886934691211,
 52.2250554994604,
 73.66496034515858,
 168.913689490089,
 170.77298495904287,
 172.52449256432055,
 66.89709612745101,
 27.056418683433666,
 30.868929748544517]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 864 µs, sys: 0 ns, total: 864 µs
Wall time: 870 µs


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,145,333.698552,288.034937,66,15,79,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.393699e+06,...,68.352148,14.895784,79,0 days 00:00:00,108.538869,13.245867,34.567482,187.081946,True,0
2,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,68.773477,15.900718,118,0 days 00:00:00,52.225055,13.245867,34.567482,127.366079,True,0
3,1,55,291.000000,279.391287,3,10,184,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,68.747008,14.775128,176,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0
4,1,92,362.373711,215.778017,21,10,202,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.422374e+06,...,68.137183,13.137425,192,0 days 00:00:00,168.913689,12.974534,35.577388,35.824467,False,10


In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_all = track[ mask_lat]
close_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
8,1,202,259.008332,324.702713,3,20,270,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.319008e+06,...,68.998871,15.950268,244,0 days 00:00:00,27.056419,12.974534,35.577388,57.709741,False,15
9,2,132,262.754401,325.077747,16,15,389,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,68.964541,15.949893,244,0 days 00:05:00,30.868930,12.780750,34.063904,58.704837,False,10
10,3,131,266.731982,325.601422,8,15,536,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,68.927977,15.952552,244,0 days 00:10:00,34.924637,12.708913,36.584485,33.831403,False,5
11,4,145,270.335063,326.000000,2,15,693,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,68.894920,15.953109,244,0 days 00:15:00,38.597920,12.845599,38.092506,17.910107,True,0
12,1,203,271.000000,318.000000,1,20,271,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.331000e+06,...,68.896332,15.756829,245,0 days 00:00:00,39.618205,12.974534,35.577388,238.807408,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,293,106,246.713605,313.000000,4,20,35137,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,69.121197,15.694082,12209,0 days 00:05:00,18.105814,15.036292,38.598690,95.519201,True,0
2403,292,122,270.000000,314.287561,2,20,35085,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908776,15.668897,12210,0 days 00:00:00,39.316062,14.985573,38.596077,26.866400,True,0
2405,294,73,274.392543,289.651057,4,15,35168,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,68.889895,15.058896,12235,0 days 00:00:00,54.214501,15.196207,39.099655,17.910596,False,5
2406,295,80,274.600400,291.595498,7,15,35231,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.334600e+06,...,68.886437,15.105794,12235,0 days 00:05:00,53.226607,15.270663,40.109338,57.712889,True,0


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_all['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_all[close_domain_all['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 180 ms, sys: 0 ns, total: 180 ms
Wall time: 414 ms


In [9]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_all['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_39155/815171471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_all['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values


In [10]:
close_domain_all = close_domain_all.reset_index(drop=True)

In [11]:
close_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-close
0,1,202,259.008332,324.702713,3,20,270,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.319008e+06,...,15.950268,244,0 days 00:00:00,27.056419,12.974534,35.577388,57.709741,False,15,15
1,2,132,262.754401,325.077747,16,15,389,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,...,15.949893,244,0 days 00:05:00,30.868930,12.780750,34.063904,58.704837,False,10,10
2,3,131,266.731982,325.601422,8,15,536,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,...,15.952552,244,0 days 00:10:00,34.924637,12.708913,36.584485,33.831403,False,5,5
3,4,145,270.335063,326.000000,2,15,693,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,...,15.953109,244,0 days 00:15:00,38.597920,12.845599,38.092506,17.910107,True,0,0
4,1,203,271.000000,318.000000,1,20,271,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.331000e+06,...,15.756829,245,0 days 00:00:00,39.618205,12.974534,35.577388,238.807408,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,293,106,246.713605,313.000000,4,20,35137,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,15.694082,12209,0 days 00:05:00,18.105814,15.036292,38.598690,95.519201,True,0,0
423,292,122,270.000000,314.287561,2,20,35085,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,15.668897,12210,0 days 00:00:00,39.316062,14.985573,38.596077,26.866400,True,0,0
424,294,73,274.392543,289.651057,4,15,35168,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,15.058896,12235,0 days 00:00:00,54.214501,15.196207,39.099655,17.910596,False,5,5
425,295,80,274.600400,291.595498,7,15,35231,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.334600e+06,...,15.105794,12235,0 days 00:05:00,53.226607,15.270663,40.109338,57.712889,True,0,0


In [12]:
len(close_domain_all)

427

# distant

In [13]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_all= track[ mask_lat ]
distant_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,68.422543,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0
1,0,145,333.698552,288.034937,66,15,79,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.393699e+06,...,68.352148,14.895784,79,0 days 00:00:00,108.538869,13.245867,34.567482,187.081946,True,0
2,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,68.773477,15.900718,118,0 days 00:00:00,52.225055,13.245867,34.567482,127.366079,True,0
3,1,55,291.000000,279.391287,3,10,184,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,68.747008,14.775128,176,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0
4,1,92,362.373711,215.778017,21,10,202,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.422374e+06,...,68.137183,13.137425,192,0 days 00:00:00,168.913689,12.974534,35.577388,35.824467,False,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,293,106,246.713605,313.000000,4,20,35137,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,69.121197,15.694082,12209,0 days 00:05:00,18.105814,15.036292,38.598690,95.519201,True,0
2403,292,122,270.000000,314.287561,2,20,35085,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908776,15.668897,12210,0 days 00:00:00,39.316062,14.985573,38.596077,26.866400,True,0
2405,294,73,274.392543,289.651057,4,15,35168,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,68.889895,15.058896,12235,0 days 00:00:00,54.214501,15.196207,39.099655,17.910596,False,5
2406,295,80,274.600400,291.595498,7,15,35231,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.334600e+06,...,68.886437,15.105794,12235,0 days 00:05:00,53.226607,15.270663,40.109338,57.712889,True,0


In [14]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_all['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_all[distant_domain_all['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 548 ms, sys: 2.81 ms, total: 551 ms
Wall time: 578 ms


In [15]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_all['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_39155/2406979236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_all['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values


In [16]:
distant_domain_all = distant_domain_all.reset_index(drop=True)


In [17]:
distant_domain_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-distant
0,0,67,326.578302,280.675950,5,10,42,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,...,14.734897,42,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True,0,0
1,0,145,333.698552,288.034937,66,15,79,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.393699e+06,...,14.895784,79,0 days 00:00:00,108.538869,13.245867,34.567482,187.081946,True,0,0
2,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,15.900718,118,0 days 00:00:00,52.225055,13.245867,34.567482,127.366079,True,0,0
3,1,55,291.000000,279.391287,3,10,184,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.351000e+06,...,14.775128,176,0 days 00:00:00,73.664960,12.974534,35.577388,NaN,True,0,0
4,1,92,362.373711,215.778017,21,10,202,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.422374e+06,...,13.137425,192,0 days 00:00:00,168.913689,12.974534,35.577388,35.824467,False,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,293,106,246.713605,313.000000,4,20,35137,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,15.694082,12209,0 days 00:05:00,18.105814,15.036292,38.598690,95.519201,True,0,0
1394,292,122,270.000000,314.287561,2,20,35085,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,15.668897,12210,0 days 00:00:00,39.316062,14.985573,38.596077,26.866400,True,0,0
1395,294,73,274.392543,289.651057,4,15,35168,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.334393e+06,...,15.058896,12235,0 days 00:00:00,54.214501,15.196207,39.099655,17.910596,False,5,5
1396,295,80,274.600400,291.595498,7,15,35231,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.334600e+06,...,15.105794,12235,0 days 00:05:00,53.226607,15.270663,40.109338,57.712889,True,0,0


In [18]:
len(distant_domain_all)

1398

In [19]:
len(close_domain_all)

427

In [20]:
len(track)

2416

In [21]:
len(close_domain_all) + len(distant_domain_all)

1825

In [22]:
distant_domain_all['latitude'].max(), distant_domain_all['latitude'].min(),close_domain_all['latitude'].max(), close_domain_all['latitude'].min()

(69.67047798270133, 67.89196414054493, 69.67047798270133, 68.78925704666102)

In [23]:
distant_domain_all['longitude'].max()

15.994079485159777

In [24]:
close_domain_all.to_csv('../saved-files/threshold-all/close-domain-threshold-all-final.csv', index=False)
distant_domain_all.to_csv('../saved-files/threshold-all/distant-domain-threshold-all-final.csv', index=False)

# END

In [ ]:
# ../saved-files/threshold-all/all-cells-first_landfall-threshold-all.csv